#### **1. 设置环境**

将 src 目录加入 Python 的搜索路径，以便导入语义标注模块

In [1]:
import os
import sys
import json
import argparse
import pandas as pd
from tqdm import tqdm

# 将上级目录加入系统路径
sys.path.append(os.path.abspath('..'))
from src.utils import load_multiver_corpus
from src.annotator.semantic_annotation import SemanticAnnotator

#### **2. 读取语料**

使用 src/utils.py 中的函数 load_multiver_corpus，加载多版本平行语料

In [32]:
# 指定数据文件路径
data_file = '../data/output/0_mt_generation.jsonl'

print(f"读取数据文件: {data_file} ...")
data = load_multiver_corpus(data_file)
print(f"成功读取数据：共 {len(data)} 条数据")

# 设置 LIMIT=10，仅读取前 10 条数据
# 设置 LIMIT=None，读取全部数据
#LIMIT = 5
LIMIT = None
if LIMIT:
    data = data.head(LIMIT)
print(f"准备为 {len(data)} 条数据标注语义类别")

读取数据文件: ../data/output/0_mt_generation.jsonl ...
成功读取数据：共 305 条数据
准备为 305 条数据标注语义类别


In [33]:
# 平行语料原文 source 节选自《鹿鼎记》
# 译文 targets 包括 3 个版本：
# human：人类译本（闵福德译）
# deepseek-v3.2：deepseek 译本
# qwen3-max：qwen 译本

# 预览数据
print(f"数据前 5 行如下：")
data.head()

数据前 5 行如下：


,id,source,targets
0,0,北风如刀，满地冰霜。,"{'human': None, 'deepseek-v3.2': 'The north wi..."
1,1,江南近海滨的一条大路上，一队清兵手执刀枪，押着七辆囚车，冲风冒寒，向北而行。,{'human': 'Along a coastal road somewhere sout...
2,2,前面三辆囚车中分别监禁的是三个男子，都作书生打扮，一个是白发老者，两个是中年人。,{'human': 'In each of the first three carts a ...
3,3,后面四辆囚车中坐的是女子，最后一辆囚车中是个少妇，怀中抱着个女婴。,{'human': 'The four rear carts were occupied b...
4,4,女婴啼哭不休。 她母亲温言相呵，女婴只是大哭。,{'human': 'The little girl was crying in a con...


#### **3. 加载模型**

加载 src/annotator/semantic_annotation.py 中定义的语义标注模型

In [34]:
# 指定用于语义标注的大模型
# 可从以下模型选取：
# qwen-flash, qwen-plus, qwen3-max, glm-4.7, deepseek-v3.2
model = 'glm-4.7'

# 加载模型前，请登录阿里云百炼平台：https://bailian.console.aliyun.com/
# 申请调用大模型服务的 API 账户
# 并在 llm_corpus_annotation/.env 文件中设置 LLM_API_KEY=sk-********

# 加载中文语义标注模型
zh_annotator = SemanticAnnotator(lang='Chinese', model=model)
print(f"中文语义标注模型加载完毕：{zh_annotator.model}")

# 加载英文语义标注模型
en_annotator = SemanticAnnotator(lang='English', model=model)
print(f"英文语义标注模型加载完毕：{en_annotator.model}")

中文语义标注模型加载完毕：glm-4.7
英文语义标注模型加载完毕：glm-4.7


#### **4. 批量标注**

标注中文原文和英文各译本的语义类别，并以 JSON 格式返回标注结果

In [35]:
def annotate_data(data, zh_annotator, en_annotator):
    results = []
    for index, row in tqdm(data.iterrows(), total=len(data), desc="Semantic Tagging"):
        try:
            # 解析数据
            record_id = row['id']
            zh_text = row['source']
            targets_dict = row['targets'] # {'human':..., 'deepseek-v3.2':...}
            
            # 标注中文原文
            if zh_text and isinstance(zh_text, str):
                source_anno = zh_annotator.annotate(zh_text)
            else:
                source_anno = None

            # 标注英文译文
            target_annos = {}
            
            for version_name, en_text in targets_dict.items():
                if en_text and isinstance(en_text, str):
                    # 分别标注各版本译文
                    en_anno = en_annotator.annotate(en_text)
                    target_annos[version_name] = en_anno
                else:
                    target_annos[version_name] = None

            # 合并中英文标注结果
            record = {
                "id": f"{record_id:06d}",
                "annotations": {
                    "source_zh": {
                        "raw_text": zh_text,
                        "usas_tags": source_anno
                    },
                    "targets_en": {
                        ver: {
                            "raw_text": txt,
                            "usas_tags": tags
                        } 
                        for ver, txt, tags in [
                            (v, targets_dict.get(v), target_annos.get(v)) 
                            for v in targets_dict.keys()
                        ]
                    }
                }
            }
            
            results.append(record)
            
        except Exception as e:
            print(f"Error at index {index}: {e}")
            continue

    return results

In [36]:
# 注意：
# 为节省 API 调用成本，本程序将大模型生成内容保存于本地缓存 data/llm_cache
# 完成首次调用后，再次调用只需从本地数据库读取生成结果
# 因此，程序运行时间显示为 0.0 秒

# 首次调用 API 翻译 305 句原文
# 生成 2 个大模型译本（deepseek + qwen3-max）
# 大约需要 20 分钟

# 开始标注
results = annotate_data(data, zh_annotator, en_annotator)

Semantic Tagging: 100%|████████████████████████████████████████████████████████████| 305/305 [4:53:13<00:00, 57.68s/it]


#### **5. 保存结果**

将大模型译文保存于指定的文件中

In [39]:
# save_results 函数：
# 将 results 列表转换为 JSON 格式
# 保存于 out_file 文件
def save_results(results, out_file):
     with open(out_file, "wt", encoding="utf-8") as fout:
        for record in results:
            fout.write(json.dumps(record, ensure_ascii=False) + "\n")

In [40]:
# 指定输出文件
out_file = '../data/output/2_semantic_annotation.jsonl'

# 保存大模型译文
save_results(results, out_file)
print(f"结果已保存至 {out_file}\n")

结果已保存至 ../data/output/2_semantic_annoation.jsonl

